In [1]:
from sampo.generator.base import SimpleSynthetic
from sampo.generator.types import SyntheticGraphType

# 1. Graph generation

In [2]:
# SimpleSynthetic object used for the simple work graph structure generation

r_seed = 231
ss = SimpleSynthetic(r_seed)

In [3]:
# simple graph
# should generate general (average) type of graph with 10 clusters from 100 to 200 vertices each

simple_wg = ss.work_graph(mode=SyntheticGraphType.GENERAL,
                          cluster_counts=10,
                          bottom_border=100,
                          top_border=200)

In [4]:
# complex graph
# should generate general (average) type of graph with 300 unique works, 100 resources and 2000 vertices

advanced_wg = ss.advanced_work_graph(works_count_top_border=2000,
                                     uniq_works=300,
                                     uniq_resources=100)

# 2. Contractor generation

In [5]:
from uuid import uuid4
from sampo.schemas.resources import Worker
from sampo.schemas.contractor import Contractor

### Manual generation
To create contractor, you should provide minimal info: unique id, contractor name, and supplied workers (simple renewable resources).

In [6]:
contractors = [
    Contractor(id=str(uuid4()),
               name="OOO Berezka",
               workers={'worker' : Worker(id='0', name='worker', count=100)})
]

### Generation from graph

In [7]:
from sampo.generator.environment.contractor_by_wg import get_contractor_by_wg

contractors = [get_contractor_by_wg(simple_wg)]

# 3. Scheduling

### Scheduler construction
Before scheduling you should specify scheduling algorithm used for transforming input data to the final schedule.
At this time SAMPO contains heuristic algorithms, such as HEFTAddEnd, HEFTAddBetween and Topological scheduler, and the Genetic algorithm.
While creation, you can specify the hyperparameters to fit the algorithm.

In [8]:
from sampo.scheduler.heft.base import HEFTScheduler

# here we can just create simple heuristic scheduler
scheduler = HEFTScheduler()

### Scheduling process
SAMPO provides a simple interface to all its features.
It called SchedulingPipeline.
Using it you only should pass all the scheduling arguments, it remembers them, and you can produce schedules in many ways.

In [9]:
from sampo.pipeline import SchedulingPipeline

project = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

project.schedule.execution_time

1194

In [10]:
project.dump('.', 'project_test')